In [1]:
import pickle

In [2]:
with open("encoded_corpus.pickle", "rb") as f:
    data = pickle.load(f)

In [3]:
X_english, X_spanish = data["X_english"], data["X_spanish"]
word2index, index2word  = data["word2index"], data["index2word"]
line_lengths_spanish, line_lengths_english = data["line_lengths_spanish"], data["line_lengths_english"]
max_length = data["max_length"]

In [4]:
EOS_token = word2index["<EOS>"]
SOS_token = word2index["<SOS>"]

In [5]:
X_english.shape, X_spanish.shape

((1944893, 90), (1944893, 90))

In [6]:
###
# Implementing PyTorch seq2seq tutorial from
# http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
###

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=max_length):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
teacher_forcing_ratio = 0.5


def train(
        input_tensor,
        target_tensor,
        encoder,
        decoder,
        encoder_optimizer,
        decoder_optimizer,
        criterion,
        max_length=max_length):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [11]:
# Helper for timing progress

import time
import math


def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [13]:
def random_example_tensors(n):
    sentence_indices = np.arange(len(X_english))
    np.random.shuffle(sentence_indices)
    training_indices = sentence_indices[:n]
    english_tensors = []
    spanish_tensors = []
    for idx in training_indices:
        english_len = line_lengths_english[idx]
        english_row = X_english[idx, :english_len]
        english_tensors.append(torch.tensor(english_row, dtype=torch.long, device=device).view(-1, 1))
        spanish_len = line_lengths_english[idx]
        spanish_row = X_spanish[idx, :spanish_len]
        spanish_tensors.append(torch.tensor(spanish_row, dtype=torch.long, device=device).view(-1, 1))
    return english_tensors, spanish_tensors

In [14]:
def train_loop(encoder, decoder, n_iters=10**5, print_every=10**3, plot_every=10, learning_rate=0.01):
    start_time = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    english_tensors, spanish_tensors = random_example_tensors(n_iters)
    
    criterion = nn.NLLLoss()

    for i, (english_tensor, spanish_tensor) in enumerate(zip(english_tensors, spanish_tensors)):
        loss = train(english_tensor, spanish_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s idx=%d (%0.4f%%) loss=%.4f' % (
                as_minutes(time.time() - start_time),
                i,
                i / n_iters * 100.0, 
                print_loss_avg))

        if i % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses)

In [40]:
hidden_size = 64
n_words = len(word2index)
encoder = EncoderRNN(n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, n_words).to(device)



In [ ]:
train_loop(encoder=encoder, decoder=decoder, n_iters=1000, print_every=20)


0m 5s idx=0 (0.0000%) loss=0.5815
1m 43s idx=20 (2.0000%) loss=10.5548
3m 38s idx=40 (4.0000%) loss=9.0180
5m 7s idx=60 (6.0000%) loss=8.0356


In [ ]:
from normalization import tokenize, detokenize

def encode_english(sentence):
    tokens = tokenize(sentence)
    return [word2index.get(word, 0) for word in tokens]

def tensor_from_sentence(sentence):
    indices = encode_english(sentence)
    return torch.tensor(indices, dtype=torch.long, device=device).view(-1, 1)

def evaluate(encoder, decoder, input_tensor, max_length=max_length):
    with torch.no_grad():
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_word_indices = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_word_indices.append(EOS_token)
                break
            else:
                decoded_word_indices.append(topi.item())
            decoder_input = topi.squeeze().detach()
        return decoded_word_indices, decoder_attentions[:di + 1]

def index_array_to_string(indices):
    print(indices)

    tokens = []
    for i in indices:
        tokens.append(index2word.get(i, "<UNK>"))
    return detokenize(tokens)
    
def tensor_to_string(tensor):
    array = tensor.data.cpu().numpy().flatten()
    return index_array_to_string(array)
    
def evaluate_randomly(encoder, decoder, n=10):
    english_tensors, spanish_tensors = random_example_tensors(n)
    for english_tensor, spanish_tensor in zip(english_tensors, spanish_tensors):
        input_sentence = tensor_to_string(english_tensor)
        expected_sentence = tensor_to_string(spanish_tensor)
        output_word_indices, attentions = evaluate(encoder, decoder, english_tensor)
        output_sentence = index_array_to_string(output_word_indices)
        print("INPUT: %s\nEXPECTED: %s\nOUTPUT: %s" % (
            input_sentence,
            expected_sentence,
            output_sentence))
        print("---")

In [ ]:
evaluate_randomly(encoder, decoder)